In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10666,2024-05-02,Israel Liga Leumit,15:00,Ironi Nahariya,Maccabi Raanana,1.34,3.08,171.5,1.86,1.84,0.0,0.00,0.00,SdLZhLHa,0.746269,0.324675,0.537634,0.543478,0.070944,148.486,41.937563,0.282434,2.4,1.341641,0.559017,186.30,1.656,0.510764,0.308432,49.0,144.802,24.403873,0.168533,0.0,0.000000,NaN,147.90,1.770,0.296395,0.167455,-51.0,90,87,2.07,1.70,125.752,178.146,0.556727,0.007644,NaN,0.42,0.084,4.047619,0.722441,0.7,-0.022441,-0.18,-0.036,-57.777778,0.000000,0.0,0.000000
10667,2024-05-02,Paraguai Lnb,20:30,Colonias Gold,Libertad,1.09,6.57,153.5,1.87,1.83,0.0,0.00,0.00,W4AmJTr4,0.917431,0.152207,0.534759,0.546448,0.069638,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,80,0,1.01,0.00,0.000,0.000,1.011735,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10668,2024-05-02,Itália Série A1 Feminina,14:00,Battipaglia F,Faenza F,2.26,1.59,141.5,1.85,1.85,0.0,0.00,0.00,WhGgZoyq,0.442478,0.628931,0.540541,0.540541,0.071409,460.750,292.601352,0.635054,1.2,1.643168,1.369306,222.42,6.226,3.414620,0.548445,-7.0,235.380,89.896593,0.381921,0.6,1.341641,2.236068,95.46,3.714,2.022803,0.544643,-28.0,66,74,3.37,1.29,454.850,367.270,0.246110,0.000000,NaN,13.41,2.682,0.469799,0.000000,0.0,0.000000,-5.00,-1.000,-0.590000,0.000000,0.0,0.000000
10669,2024-05-02,Letônia Lbl,13:00,Ventspils,Rigas Zelli,1.42,2.78,174.5,1.87,1.79,0.0,0.00,0.00,M1Rb3vLl,0.704225,0.359712,0.534759,0.558659,0.063938,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,123.072,15.681236,0.127415,1.8,1.643168,0.912871,143.50,1.424,0.243680,0.171124,30.0,103,82,2.01,1.75,0.000,0.000,0.457936,0.030912,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10670,2024-05-02,Lituânia Lkl,12:15,Lietkabelis,Mazeikiai,1.19,4.32,164.5,1.85,1.85,0.0,0.00,0.00,bJMmYuQ9,0.840336,0.231481,0.540541,0.540541,0.071818,182.846,87.909158,0.480783,0.6,1.341641,2.236068,105.30,2.124,0.969319,0.456365,-9.0,345.718,274.394823,0.793696,0.6,1.341641,2.236068,788.80,4.724,4.080665,0.863816,-70.0,78,68,1.35,11.60,144.644,392.464,0.803355,0.000000,NaN,-2.72,-0.544,-0.349265,0.657427,0.4,-0.257427,-2.85,-0.570,-5.824561,0.000000,0.0,0.000000
10671,2024-05-02,Lituânia Lkl,12:50,Zalgiris Kaunas,Nevezis,1.02,11.60,167.5,1.85,1.85,0.0,0.00,0.00,htMiXauG,0.980392,0.086207,0.540541,0.540541,0.066599,140.036,44.590107,0.318419,1.2,1.643168,1.369306,96.90,1.796,0.947143,0.527363,34.0,285.748,163.881774,0.573519,1.2,1.643168,1.369306,249.40,3.328,2.068434,0.621525,-1.0,95,116,1.02,2.15,133.608,335.578,1.185609,0.000000,NaN,0.75,0.150,0.133333,0.709764,0.8,0.090236,-1.54,-0.308,-34.415584,0.000000,0.0,0.000000
10672,2024-05-02,México Cibacopa,00:15,Caballeros de Culiacan,Halcones de Obregon,2.12,1.67,181.5,1.80,1.86,1.5,2.00,1.51,YcE58fFG,0.471698,0.598802,0.555556,0.537634,0.070501,440.914,182.079947,0.412960,0.6,1.341641,2.236068,193.13,5.432,2.054889,0.378293,-54.0,164.172,29.190900,0.177807,1.8,1.643168,0.912871,141.04,1.658,0.339662,0.204862,39.0,89,86,2.17,1.64,182.178,152.006,0.167915,0.023184,0.197426,-0.80,-0.160,-7.000000,0.478842,0.2,-0.278842,-5.00,-1.000,-0.670000,0.487581,0.2,-0.287581
10673,2024-05-

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10700,13:00,Croácia Premijer Liga,Cibona,Zabok,1.65,Back Home
